In [2]:
from bs4 import BeautifulSoup

from html.parser import HTMLParser

import json
import urllib

def curl(url):
    req = urllib.request.Request(url)
    response = urllib.request.urlopen(req)
    data = response.read()
    return data


def get_repo_stars_and_contributors(html_content):
    # Parse the HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Array to store extracted vulnerabilities
    vulnerabilities = []

    # Find all <li> elements corresponding to vulnerabilities
    stars_items = soup.find_all('span', {'id': 'repo-stars-counter-star'})

    # Extract the required data for each item
    for item in stars_items:
        stars = item.text 

        # Find all <li> elements corresponding to vulnerabilities
    cont_items = soup.find_all('span', {'class': 'Counter ml-1'})
    for item in cont_items:
        n_contributers = item.text 
    
    return stars, n_contributers
    
def get_n_dependencies(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    vulnerabilities = []

    items = soup.find_all('div', {'class': 'd-flex flex-items-center gap-1 text-bold'})

    n_dep = '-1'
    for item in items:
        n_dep = item.text.strip()

    return n_dep.split()[0]

def get_n_dependents(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    items = soup.find_all('div', {'class': 'table-list-header-toggle states flex-auto pl-0'})

    n_repos, n_pkgs = None, None
    for item in items:
        n_deps = item.find_all('a', class_='btn-link')
        for i, n_dep in enumerate(n_deps):
            if i == 0:
                n_repos = n_dep.text.strip().split()[0]
            else:
                n_pkgs = n_dep.text.strip().split()[0]

    return n_repos, n_pkgs

def get_tags_desc_from_github(github_content):
    soup = BeautifulSoup(github_content, 'html.parser')
    result = []
    
    # Find the div with the specific class for the "About" section
    about_div = soup.find('div', class_='about-margin')
    # print(about_div.find('div', class_='BorderGrid-row'))
    divs = about_div.find_all('div', class_='BorderGrid-cell')

    
    about_text = None
    tags = []
    
    # if about_div:
    for i, border_view in enumerate(divs):
        # print('xxxx', border_view)
        if i == 0:
            about_text = border_view.find('p', class_='my-3').text.strip()
            break

    topics = about_div.find_all('a', class_='topic-tag')
    for i, topic in enumerate(topics):
        tags.append(topic.text.strip())

    # href = a_tag['href']
    # if 'https://github.com' in href and 'github.com/snyk' not in href.lower():
    #     repo_link = get_repo_link(href)
    #     if repo_link:
    #         github_links.add(repo_link)
    
    return about_text, tags

def get_git_data(git_url, only_contributors=False):
    stars, contrs, n_dependencies, n_dependents, desc, tags = None, None, None, None, None, []
    
    try: 
        html_content = curl(git_url)
        desc, tags = get_tags_desc_from_github(html_content)
        stars, contrs = get_repo_stars_and_contributors(html_content)
    except:
        print(f'Error in stars or tags for {git_url}')
        return stars, contrs, n_dependencies, n_dependents, desc, tags

    if only_contributors:
        return stars, contrs, n_dependencies, n_dependents, desc, tags

    if git_url[-1] == '/':
        git_url = git_url[:-1]

    try:
        dependencies_url = f'{git_url}/network/dependencies'
        html_content = curl(dependencies_url)
        n_dependencies = get_n_dependencies(html_content)
    except:
        print(f'Error in n_dependencies for {git_url}')
        return stars, contrs, n_dependencies, n_dependents, desc, tags

    try:
        dependents_url = f'{git_url}/network/dependents'
        html_content = curl(dependents_url)
        n_dependents = get_n_dependents((html_content))
    except:
        print(f'Error in n_dependents for {git_url}')
        return stars, contrs, n_dependencies, n_dependents, desc, tags

    return stars, contrs, n_dependencies, n_dependents, desc, tags


res = get_git_data('https://github.com/serilog-contrib/serilog-enrichers-globallogcontext', True)
print(res)

('29', '2', None, None, 'A Serilog Enricher for adding properties to all log events in your app', ['serilog', 'enricher', 'hacktoberfest', 'global-context', 'enrichers', 'augustoproiete', 'augusto-proiete', 'globallogcontext', 'global-log-context', 'globalcontext', 'logcontext', 'log-context'])


In [ ]:
# Function to convert star strings to numbers
def convert_k_to_numeric(val):
    if pd.isna(val):
        return None
    if 'k' in val:
        return float(val.replace('k', '')) * 1000
    return float(val)

# Function to process dependents (split into dependent packages and dependent repos)
def process_dependents(val):
    # print(val)
    if pd.isna(val):
        return None
    if '?' in val:
        return None
    
    val = eval(val)
    dep_packages, dep_repos = val
    # Convert to numeric after removing commas
    dep_packages = int(dep_packages.replace(',', '')) if dep_packages != '0' else 0
    dep_repos = int(dep_repos.replace(',', '')) if dep_repos != '0' else 0
    return dep_packages

def process_dependencies(val):
    if pd.isna(val):
        return None
    # val = eval(val)
    deps = int(val.replace(',', '')) if val != '0' else 0
    return deps

def process_contributors(val):
    if pd.isna(val):
        return None
    # if val[-1] == '+':
    #     val = val[:-1]
    # val = eval(val)
    deps = int(val) if val != '0' else 0
    return deps


# process_dependencies('1,200')
# process_dependents(('73,903', '73,903'))

# Apply conversion to Stars column
final_df['Stars'] = final_df['Stars'].apply(convert_k_to_numeric)
final_df['Dependents'] = final_df['Dependents'].apply(process_dependents)
final_df['Dependencies'] = final_df['Dependencies'].apply(process_dependencies)
final_df['Contributors'] = final_df['Contributors'].apply(process_contributors)

# final_df['Dependents_Pkgs'] = final_df['Dependents_Pkgs'].apply(process_contributors)
    

In [6]:
import pandas as pd

# final_df.to_csv(f'data/snyk_and_advisory_with_git_info_v2.csv', index=False)
final_df = pd.read_csv('../data/snyk_and_advisory_with_git_info.csv')

# final_df = pd.read_csv('data/snyk_and_advisory.csv')

final_df

,ID,Package,CWE,CVE,Date,Platform,Version Range,GitHub URL,Source,Stars,Contributors,Dependencies,Dependents,Dependents_Pkgs,Desciptions,Tags
0,SNYK-DOTNET-MAGICKNETCORE-7856102,magick.net.core,CWE-122,CVE-2023-4863,1 Sep 2024,NuGet,"['[,13.3.0)']",https://github.com/dlemstra/Magick.NET,Snyk,3.5k,38,44,"('0', '0')",NaN,NaN,NaN
1,SNYK-DOTNET-SERILOGENRICHERSCLIENTINFO-7853254,serilog.enrichers.clientinfo,CWE-348,CVE-2024-44930,30 Aug 2024,NuGet,"['[,2.1.0)']",https://github.com/serilog-contrib/serilog-enr...,Snyk,102,16,15,"('731', '731')",NaN,NaN,NaN
2,SNYK-DOTNET-RESTSHARP-7853196,restsharp,CWE-113,CVE-2024-45302,30 Aug 2024,NuGet,"['[107.0.0-preview.1,112.0.0)']",https://github.com/restsharp/RestSharp,Snyk,9.7k,271,"1,220","('73,859', '73,859')",NaN,NaN,NaN
3,SNYK-DOTNET-BOUNCYCASTLE-7831269,bouncycastle,CWE-835,CVE-2024-30172,26 Aug 2024,NuGet,"['[0,]']",NaN,Snyk,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SNYK-DOTNET-BOUNCYCASTLECRYPTOGRAPHY-7831268,bouncycastle.cryptography,CWE-835,CVE-2024-30172,26 Aug 2024,NuGet,"['[,2.3.1)']",NaN,Snyk,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28148,GHSA-2pj2-gchf-wmw7,net.lingala.zip4j:zip4j,CWE-346,CVE-2023-22899,10 Jan 2023,Maven,"[0,2.11.3)",https://github.com/srikanth-lingala/zip4j,Git,2.1k,31,14,"('0', '0')",NaN,NaN,NaN
28149,GHSA-2x48-p6cq-5xcw,github.com/go-sonic/sonic,CWE-22,CVE-2022-46959,23 Jan 2023,Go,"[0,1.0.5)",https://github.com/go-sonic/sonic,Git,2k,16,128,"('1', '1')",NaN,NaN,NaN
28150,GHSA-3244-8mff-w398,github.com/gotify/server,CWE-79,NaN,10 Jan 2023,Go,"[0,2.2.3)",https://github.com/gotify/server,Git,11.9k,38,"1,769","('5', '5')",NaN,NaN,NaN
28151,GHSA-33gv-rvgq-gpxp,BTCPayServer.Client,CWE-74,CVE-2023-0493,27 Jan 2023,NuGet,"[0,1.7.5)",https://github.com/btcpayserver/btcpayserver,Git,6.6k,166,60,"('17', '17')",NaN,NaN,NaN


# Update Missing Values

In [ ]:
from time import sleep
from tqdm import tqdm  # Import tqdm for the progress bar


git_crawl_cache = {}

# Add empty columns to hold stars, dependencies, and dependents
final_df['Stars'] = None
final_df['Contributors'] = None
final_df['Dependencies'] = None
final_df['Dependents'] = None
final_df['Dependents_Pkgs'] = None
final_df['Desciptions'] = None
final_df['Tags'] = None


def fill_git_data(base_df):
    df = base_df.copy()
    # Loop through each row in the DataFrame and apply the get_git_data function
    for idx, row in tqdm(df.iterrows(), total=final_df.shape[0], desc="Processing rows", unit="row"):
        # i+= 1
        # if i == 10:
        #     break
        git_url = row['GitHub URL']
        if pd.isna(git_url):
            continue
        # if pd.notna(git_url):
        #     try:
        #         dependents_url = f'{git_url}/network/dependents'
        #         html_content = curl(dependents_url)
        #         n_dependents = get_n_dependents((html_content))
        #         n_dep_pkgs = n_dependents[1]
        #         final_df.at[idx, 'Dependents_Pkgs'] = n_dep_pkgs
        #     except:
        #         pass

        if git_url in git_crawl_cache:    
            stars, contrs, dependencies, dependents, desc, tags = get_git_data(git_url)
        elif git_url in final_df_old_with_git['GitHub URL'].values:
            res = get_git_data(git_url, True)
            stars, contrs, dependencies, dependents, desc, tags = res
            matched_row = final_df_old_with_git[final_df_old_with_git['GitHub URL'] == row['GitHub URL']].iloc[0]# Update final_df with values from final_df_old_with_git
            df.at[idx, 'Stars'] = stars
            df.at[idx, 'Contributors'] = contrs
            df.at[idx, 'Dependencies'] = matched_row['Dependencies']
            df.at[idx, 'Dependents'] = matched_row['Dependents']
            # final_df.at[idx, 'Desciptions'] = matched_row['Desciptions']
            # final_df.at[idx, 'Tags'] = matched_row['Tags']
        else:
            if pd.notna(git_url):
                res = get_git_data(git_url)
                stars, contrs, dependencies, dependents, desc, tags = res
                # print(res)
                git_crawl_cache[git_url] = res
                
                df.at[idx, 'Stars'] = stars
                df.at[idx, 'Contributors'] = contrs
                df.at[idx, 'Dependencies'] = dependencies
                df.at[idx, 'Dependents'] = dependents
                df.at[idx, 'Desciptions'] = desc
                df.at[idx, 'Tags'] = str(tags)
                
                

            sleep(0.01)
    return df
    # Display the updated DataFrame
final_df = fill_git_data(final_df)
final_df

# final_df.to_csv(f'../data/snyk_and_advisory_with_git_info_v3.csv', index=False)
